In [2]:
# To support both python 2 and python 3
from __future__ import division, print_function, unicode_literals

try:
    import tensorflow.compat.v1 as tf
    tf.disable_v2_behavior()
except ImportError:
    import tensorflow as tf

# Common imports
import numpy as np
import os

from tensorflow_graph_in_jupyter import show_graph
from sklearn.datasets import fetch_california_housing
from sklearn.preprocessing import StandardScaler
import tfgraphviz as tfg

%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt

In [3]:
print(tf.__version__)

2.1.0


In [4]:
import numpy as np
import os
import tarfile
import joblib
from sklearn.datasets.base import _pkl_filepath, get_data_home

archive_path = "cal_housing.tgz" # change the path if it's not in the current directory
data_home = get_data_home(data_home=None) # change data_home if you are not using ~/scikit_learn_data
if not os.path.exists(data_home):
    os.makedirs(data_home)
filepath = _pkl_filepath(data_home, 'cal_housing.pkz')

with tarfile.open(mode="r:gz", name=archive_path) as f:
    cal_housing = np.loadtxt(
        f.extractfile('CaliforniaHousing/cal_housing.data'),
        delimiter=',')
    # Columns are not in the same order compared to the previous
    # URL resource on lib.stat.cmu.edu
    columns_index = [8, 7, 2, 3, 4, 5, 6, 1, 0]
    cal_housing = cal_housing[:, columns_index]

    joblib.dump(cal_housing, filepath, compress=6)

C:\Users\felipe.figueiredo\Anaconda3\envs\ML\lib\site-packages\sklearn\utils\deprecation.py:143: FutureWarning: The sklearn.datasets.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.datasets. Anything that cannot be imported from sklearn.datasets is now part of the private API.
  warnings.warn(message, FutureWarning)


In [5]:
housing = fetch_california_housing()
m, n = housing.data.shape

scaler = StandardScaler()
scaled_housing_data = scaler.fit_transform(housing.data)
scaled_housing_data_plus_bias = np.c_[np.ones((m, 1)), scaled_housing_data]

In [6]:
# to make this notebook's output stable across runs
def reset_graph(seed=42):
    tf.reset_default_graph()
    tf.set_random_seed(seed)
    np.random.seed(seed)

In [7]:
n_epochs = 1000
learning_rate = 0.01

In [8]:
reset_graph()

X = tf.constant(scaled_housing_data_plus_bias, dtype=tf.float32, name="X")
y = tf.constant(housing.target.reshape(-1, 1), dtype=tf.float32, name="y")
theta = tf.Variable(tf.random_uniform([n + 1, 1], -1.0, 1.0, seed=42), name="theta")
y_pred = tf.matmul(X, theta, name="predictions")
error = y_pred - y
mse = tf.reduce_mean(tf.square(error), name="mse")
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
training_op = optimizer.minimize(mse)

init = tf.global_variables_initializer()

saver = tf.train.Saver()

In [9]:
with tf.Session() as sess:
    sess.run(init)

    for epoch in range(n_epochs):
        if epoch % 100 == 0:
            print("Epoch", epoch, "MSE =", mse.eval())
            save_path = saver.save(sess, "./my_model.ckpt")
        sess.run(training_op)
    
    best_theta = theta.eval()
    save_path = saver.save(sess, "./my_model_final.ckpt")

Epoch 0 MSE = 9.16154
Epoch 100 MSE = 0.7145003
Epoch 200 MSE = 0.56670487
Epoch 300 MSE = 0.55557173
Epoch 400 MSE = 0.5488112
Epoch 500 MSE = 0.54363626
Epoch 600 MSE = 0.53962904
Epoch 700 MSE = 0.5365092
Epoch 800 MSE = 0.53406775
Epoch 900 MSE = 0.5321473


In [10]:
best_theta

array([[ 2.0685525 ],
       [ 0.8874027 ],
       [ 0.14401658],
       [-0.34770882],
       [ 0.36178368],
       [ 0.00393811],
       [-0.04269556],
       [-0.6614528 ],
       [-0.6375277 ]], dtype=float32)

In [11]:
with tf.Session() as sess:
    saver.restore(sess, "./my_model_final.ckpt")
    best_theta_restored = theta.eval()

INFO:tensorflow:Restoring parameters from ./my_model_final.ckpt


In [12]:
# Returns True if two arrays are element-wise equal within a tolerance.
np.allclose(best_theta, best_theta_restored)

True

In [13]:
saver = tf.train.Saver({"weights": theta})

In [14]:
reset_graph()
# notice that we start with an empty graph.

saver = tf.train.import_meta_graph("./my_model_final.ckpt.meta")  # this loads the graph structure
theta = tf.get_default_graph().get_tensor_by_name("theta:0")

with tf.Session() as sess:
    saver.restore(sess, "./my_model_final.ckpt")  # this restores the graph's state
    best_theta_restored = theta.eval()

INFO:tensorflow:Restoring parameters from ./my_model_final.ckpt


In [15]:
# Returns True if two arrays are element-wise equal within a tolerance.
np.allclose(best_theta, best_theta_restored)

True

In [21]:
show_graph(tf.get_default_graph())

In [19]:
# Load the TensorBoard notebook extension
%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [20]:
%tensorboard --logdir ./

Reusing TensorBoard on port 6006 (pid 17732), started 0:01:13 ago. (Use '!kill 17732' to kill it.)